# PART 1: Data Processing

In [4]:
import pandas as pd
import numpy as np
import seaborn as sns
sns.set()

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

import pickle

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 25)
pd.set_option('display.precision', 2)
pd.set_option('display.float_format', lambda x: '%.f' % x)

## Crime Data Cleansing

- Only inlcude data from 2015 and beyond
- Only include Queens, Brooklyn, and Manhattan

In [128]:
# pulling data from 2015 and beyond 

l = [] #list of index number to be skipped (<2014)
n = sum(1 for line in open(filename))-1  # Calculate number of rows in file
filename = 'Data/NYPD_Complaint_Data_Historic.csv'

for i in range(5):
    s = list(range(i*(n//5))) + list(range((i+1)*(n//5),n)) # skiprow index to parce the data by pieces
    s = [v+1 for v in s] #to account for headers
    sub_df = pd.read_csv(filename, skiprows=s) #sub_population in 5 pieces
    l.extend(sub_df[sub_df.CMPLNT_FR_DT.str[-4:]<'2014'].index + i*(n//5) + 1) #adding index of each subpieces <2012 to the list

In [129]:
df = pd.read_csv(filename, skiprows=l) # Population

In [3]:
# small population to work with (scale later)
df_s = df[(df.BORO_NM == 'QUEENS') | (df.BORO_NM == 'BROOKLYN')| (df.BORO_NM == 'MANHATTAN')]
df_s = df_s[df_s.CMPLNT_FR_DT.str[-4:] > '2015']

In [219]:
with open("df_s.pkl", 'rb') as picklefile: 
    df_s = pickle.load(picklefile)

In [220]:
# Only include key columns 
df_s = df_s.loc[:,['CMPLNT_NUM', 'CMPLNT_FR_DT', 'CMPLNT_FR_TM', 'RPT_DT', 'KY_CD', 'OFNS_DESC', 'PD_CD',
       'PD_DESC','LAW_CAT_CD', 'BORO_NM',
       'LOC_OF_OCCUR_DESC', 'PREM_TYP_DESC', 'X_COORD_CD', 'Y_COORD_CD',
       'SUSP_AGE_GROUP', 'SUSP_RACE', 'SUSP_SEX',
       'Latitude', 'Longitude', 'Lat_Lon',
       'VIC_AGE_GROUP', 'VIC_RACE', 'VIC_SEX']]

In [221]:
df_s.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1281372 entries, 2553237 to 817264
Data columns (total 23 columns):
CMPLNT_NUM           1281372 non-null int64
CMPLNT_FR_DT         1281372 non-null object
CMPLNT_FR_TM         1281372 non-null object
RPT_DT               1281372 non-null object
KY_CD                1281372 non-null int64
OFNS_DESC            1281308 non-null object
PD_CD                1281372 non-null float64
PD_DESC              1281372 non-null object
LAW_CAT_CD           1281372 non-null object
BORO_NM              1281372 non-null object
LOC_OF_OCCUR_DESC    1034958 non-null object
PREM_TYP_DESC        1276588 non-null object
X_COORD_CD           1043976 non-null float64
Y_COORD_CD           1043976 non-null float64
SUSP_AGE_GROUP       939505 non-null object
SUSP_RACE            939505 non-null object
SUSP_SEX             939505 non-null object
Latitude             1043976 non-null float64
Longitude            1043976 non-null float64
Lat_Lon              104397

In [222]:
df_s.dropna(subset = ['Lat_Lon'], inplace = True)

## Create Census Tract to Location (NTA) Mapping

In [191]:
# latlong to cen_tract 
cen_tract = pd.read_csv('Data/new-york-city-census-data/census_block_loc.csv')

In [192]:
cen_tract['BlockCode'] = cen_tract['BlockCode'].apply(lambda s : str(s)).apply(lambda s : s[:11])

In [193]:
cen_tract.nunique()

Latitude      200
Longitude     200
BlockCode    2995
County         15
State           2
dtype: int64

In [195]:
cen_tract['lookup'] = cen_tract['BlockCode'].str[2:]
cen_tract

,Latitude,Longitude,BlockCode,County,State,lookup
0,40,-74,34023007600,Middlesex,NJ,023007600
1,40,-74,34023007600,Middlesex,NJ,023007600
2,40,-74,34023007600,Middlesex,NJ,023007600
3,40,-74,34023007600,Middlesex,NJ,023007600
4,40,-74,34023007402,Middlesex,NJ,023007402
...,...,...,...,...,...,...
38391,41,-74,36119007600,Westchester,NY,119007600
38392,41,-74,36119007600,Westchester,NY,119007600
38393,41,-74,36119007600,Westchester,NY,119007600
38394,41,-74,36119007600,Westchester,NY,119007600


In [196]:
cen_tract['Latitude'] = cen_tract['Latitude'].apply(lambda x : "{:.2f}".format(x))
cen_tract['Longitude'] = cen_tract['Longitude'].apply(lambda x : "{:.2f}".format(x))
cen_tract['latlong'] = cen_tract['Latitude'].apply(lambda x: str(x)) + cen_tract['Longitude'].apply(lambda x: str(x))
cen_tract

,Latitude,Longitude,BlockCode,County,State,lookup,latlong
0,40.48,-74.28,34023007600,Middlesex,NJ,023007600,40.48-74.28
1,40.48,-74.28,34023007600,Middlesex,NJ,023007600,40.48-74.28
2,40.48,-74.27,34023007600,Middlesex,NJ,023007600,40.48-74.27
3,40.48,-74.27,34023007600,Middlesex,NJ,023007600,40.48-74.27
4,40.48,-74.27,34023007402,Middlesex,NJ,023007402,40.48-74.27
...,...,...,...,...,...,...,...
38391,40.93,-73.66,36119007600,Westchester,NY,119007600,40.93-73.66
38392,40.93,-73.66,36119007600,Westchester,NY,119007600,40.93-73.66
38393,40.93,-73.66,36119007600,Westchester,NY,119007600,40.93-73.66
38394,40.93,-73.65,36119007600,Westchester,NY,119007600,40.93-73.65


In [200]:
# nta data
nta = pd.read_excel('Data/nyc2010census_tabulation_equiv.xlsx')

In [201]:
nta.columns = ['boro', 'fips', 'boro_code', 'cen_tract', 'puma', 'code', 'nta']

In [202]:
nta.drop([0,1,2,3], inplace = True)
nta.reset_index(drop = True)

,boro,fips,boro_code,cen_tract,puma,code,nta
0,Bronx,005,2,031000,3704,BX31,Allerton-Pelham Gardens
1,Bronx,005,2,031200,3704,BX31,Allerton-Pelham Gardens
2,Bronx,005,2,031400,3704,BX31,Allerton-Pelham Gardens
3,Bronx,005,2,031600,3704,BX31,Allerton-Pelham Gardens
4,Bronx,005,2,031800,3704,BX31,Allerton-Pelham Gardens
...,...,...,...,...,...,...,...
2163,Staten Island,085,5,016901,3903,SI07,Westerleigh
2164,Staten Island,085,5,018701,3903,SI07,Westerleigh
2165,Staten Island,085,5,018901,3903,SI07,Westerleigh
2166,Staten Island,085,5,019700,3903,SI07,Westerleigh


In [203]:
nta['lookup'] = nta['fips'] + nta['cen_tract']

In [204]:
nta

,boro,fips,boro_code,cen_tract,puma,code,nta,lookup
4,Bronx,005,2,031000,3704,BX31,Allerton-Pelham Gardens,005031000
5,Bronx,005,2,031200,3704,BX31,Allerton-Pelham Gardens,005031200
6,Bronx,005,2,031400,3704,BX31,Allerton-Pelham Gardens,005031400
7,Bronx,005,2,031600,3704,BX31,Allerton-Pelham Gardens,005031600
8,Bronx,005,2,031800,3704,BX31,Allerton-Pelham Gardens,005031800
...,...,...,...,...,...,...,...,...
2167,Staten Island,085,5,016901,3903,SI07,Westerleigh,085016901
2168,Staten Island,085,5,018701,3903,SI07,Westerleigh,085018701
2169,Staten Island,085,5,018901,3903,SI07,Westerleigh,085018901
2170,Staten Island,085,5,019700,3903,SI07,Westerleigh,085019700


In [208]:
# Merge cen_track with nta

mapping_df = pd.merge(cen_tract, nta[['lookup','code','nta']], how = 'inner', left_on = 'lookup', right_on = 'lookup' )
mapping_df

,Latitude,Longitude,BlockCode,County,State,lookup,latlong,code,nta
0,40.48,-74.23,36085990100,Richmond,NY,085990100,40.48-74.23,SI99,park-cemetery-etc-Staten Island
1,40.48,-74.23,36085990100,Richmond,NY,085990100,40.48-74.23,SI99,park-cemetery-etc-Staten Island
2,40.48,-74.23,36085990100,Richmond,NY,085990100,40.48-74.23,SI99,park-cemetery-etc-Staten Island
3,40.48,-74.22,36085990100,Richmond,NY,085990100,40.48-74.22,SI99,park-cemetery-etc-Staten Island
4,40.48,-74.22,36085990100,Richmond,NY,085990100,40.48-74.22,SI99,park-cemetery-etc-Staten Island
...,...,...,...,...,...,...,...,...,...
18048,40.91,-73.90,36005031900,Bronx,NY,005031900,40.91-73.90,BX22,North Riverdale-Fieldston-Riverdale
18049,40.91,-73.92,36005031900,Bronx,NY,005031900,40.91-73.92,BX22,North Riverdale-Fieldston-Riverdale
18050,40.91,-73.91,36005031900,Bronx,NY,005031900,40.91-73.91,BX22,North Riverdale-Fieldston-Riverdale
18051,40.91,-73.91,36005031900,Bronx,NY,005031900,40.91-73.91,BX22,North Riverdale-Fieldston-Riverdale


In [209]:
mapping_df.drop_duplicates(inplace = True)

In [230]:
mapping_df.drop_duplicates(['latlong'], inplace = True)

## Merging cen_track and nta with Crime Datea (i.e. df_s)

In [223]:
def latlong(s):
    tup = tuple(float(i) for i in s.strip("()").split(","))
    return "{:.2f}".format(tup[0])+"{:.2f}".format(tup[1])

df_s['Lat_Lon'] = df_s['Lat_Lon'].apply(latlong)
df_s.head(3)

,CMPLNT_NUM,CMPLNT_FR_DT,CMPLNT_FR_TM,RPT_DT,KY_CD,OFNS_DESC,PD_CD,PD_DESC,LAW_CAT_CD,BORO_NM,LOC_OF_OCCUR_DESC,PREM_TYP_DESC,X_COORD_CD,Y_COORD_CD,SUSP_AGE_GROUP,SUSP_RACE,SUSP_SEX,Latitude,Longitude,Lat_Lon,VIC_AGE_GROUP,VIC_RACE,VIC_SEX
2553237,254010275,01/01/2016,16:05:00,01/01/2016,106,FELONY ASSAULT,106,ASSAULT POLICE/PEACE OFFICER,FELONY,QUEENS,FRONT OF,STREET,1053133,159320,18-24,BLACK,M,41,-74,40.60-73.75,25-44,WHITE,M
3678094,945307000,01/01/2016,08:00:00,07/11/2016,351,CRIMINAL MISCHIEF & RELATED OF,258,"CRIMINAL MISCHIEF 4TH, GRAFFIT",MISDEMEANOR,BROOKLYN,FRONT OF,RESTAURANT/DINER,998510,155296,NaN,NaN,NaN,41,-74,40.59-73.95,UNKNOWN,UNKNOWN,D
2944583,719701416,01/01/2016,08:00:00,12/19/2016,351,CRIMINAL MISCHIEF & RELATED OF,258,"CRIMINAL MISCHIEF 4TH, GRAFFIT",MISDEMEANOR,BROOKLYN,FRONT OF,STORE UNCLASSIFIED,1000566,156826,NaN,NaN,NaN,41,-74,40.60-73.94,UNKNOWN,UNKNOWN,D
3325623,355498796,01/01/2016,09:00:00,03/08/2016,578,HARRASSMENT 2,638,"HARASSMENT,SUBD 3,4,5",VIOLATION,QUEENS,INSIDE,RESIDENCE - APT. HOUSE,1033265,213483,UNKNOWN,WHITE,M,41,-74,40.75-73.82,45-64,ASIAN / PACIFIC ISLANDER,M
2597989,858033700,01/01/2016,00:01:00,04/08/2016,351,CRIMINAL MISCHIEF & RELATED OF,258,"CRIMINAL MISCHIEF 4TH, GRAFFIT",MISDEMEANOR,QUEENS,INSIDE,FAST FOOD,1029842,200710,UNKNOWN,UNKNOWN,U,41,-74,40.72-73.84,UNKNOWN,UNKNOWN,D
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15309,916182881,12/31/2018,04:00:00,12/31/2018,347,INTOXICATED & IMPAIRED DRIVING,905,"INTOXICATED DRIVING,ALCOHOL",MISDEMEANOR,MANHATTAN,NaN,STREET,1007320,255391,45-64,BLACK,M,41,-74,40.87-73.92,UNKNOWN,UNKNOWN,E
112896,743292142,12/31/2018,19:41:00,12/31/2018,341,PETIT LARCENY,338,"LARCENY,PETIT FROM BUILDING,UN",MISDEMEANOR,QUEENS,INSIDE,LIQUOR STORE,1015653,194856,UNKNOWN,WHITE,F,41,-74,40.70-73.89,UNKNOWN,UNKNOWN,D
245046,439692376,12/31/2018,06:20:00,12/31/2018,578,HARRASSMENT 2,638,"HARASSMENT,SUBD 3,4,5",VIOLATION,BROOKLYN,INSIDE,OTHER,1000401,192939,25-44,BLACK,M,41,-74,40.70-73.94,18-24,BLACK,M
407032,637872275,12/31/2018,20:10:00,12/31/2018,105,ROBBERY,379,"ROBBERY,GAS STATION",FELONY,QUEENS,INSIDE,GAS STATION,1050202,193549,UNKNOWN,BLACK,M,41,-74,40.70-73.76,UNKNOWN,UNKNOWN,D


In [232]:
df_s = pd.merge(df_s, mapping_df[['latlong','code','nta','BlockCode']], how = 'left', left_on = 'Lat_Lon', right_on = 'latlong' )
df_s.head(3)

,CMPLNT_NUM,CMPLNT_FR_DT,CMPLNT_FR_TM,RPT_DT,KY_CD,OFNS_DESC,PD_CD,PD_DESC,LAW_CAT_CD,BORO_NM,LOC_OF_OCCUR_DESC,PREM_TYP_DESC,X_COORD_CD,Y_COORD_CD,SUSP_AGE_GROUP,SUSP_RACE,SUSP_SEX,Latitude,Longitude,Lat_Lon,VIC_AGE_GROUP,VIC_RACE,VIC_SEX,latlong,code,nta,BlockCode
0,254010275,01/01/2016,16:05:00,01/01/2016,106,FELONY ASSAULT,106,ASSAULT POLICE/PEACE OFFICER,FELONY,QUEENS,FRONT OF,STREET,1053133,159320,18-24,BLACK,M,41,-74,40.60-73.75,25-44,WHITE,M,40.60-73.75,QN15,Far Rockaway-Bayswater,36081101001
1,945307000,01/01/2016,08:00:00,07/11/2016,351,CRIMINAL MISCHIEF & RELATED OF,258,"CRIMINAL MISCHIEF 4TH, GRAFFIT",MISDEMEANOR,BROOKLYN,FRONT OF,RESTAURANT/DINER,998510,155296,NaN,NaN,NaN,41,-74,40.59-73.95,UNKNOWN,UNKNOWN,D,40.59-73.95,BK17,Sheepshead Bay-Gerritsen Beach-Manhattan Beach,36047060600
2,719701416,01/01/2016,08:00:00,12/19/2016,351,CRIMINAL MISCHIEF & RELATED OF,258,"CRIMINAL MISCHIEF 4TH, GRAFFIT",MISDEMEANOR,BROOKLYN,FRONT OF,STORE UNCLASSIFIED,1000566,156826,NaN,NaN,NaN,41,-74,40.60-73.94,UNKNOWN,UNKNOWN,D,40.60-73.94,BK17,Sheepshead Bay-Gerritsen Beach-Manhattan Beach,36047059402
3,355498796,01/01/2016,09:00:00,03/08/2016,578,HARRASSMENT 2,638,"HARASSMENT,SUBD 3,4,5",VIOLATION,QUEENS,INSIDE,RESIDENCE - APT. HOUSE,1033265,213483,UNKNOWN,WHITE,M,41,-74,40.75-73.82,45-64,ASIAN / PACIFIC ISLANDER,M,40.75-73.82,QN62,Queensboro Hill,36081083700
4,858033700,01/01/2016,00:01:00,04/08/2016,351,CRIMINAL MISCHIEF & RELATED OF,258,"CRIMINAL MISCHIEF 4TH, GRAFFIT",MISDEMEANOR,QUEENS,INSIDE,FAST FOOD,1029842,200710,UNKNOWN,UNKNOWN,U,41,-74,40.72-73.84,UNKNOWN,UNKNOWN,D,40.72-73.84,QN17,Forest Hills,36081074900
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1043971,916182881,12/31/2018,04:00:00,12/31/2018,347,INTOXICATED & IMPAIRED DRIVING,905,"INTOXICATED DRIVING,ALCOHOL",MISDEMEANOR,MANHATTAN,NaN,STREET,1007320,255391,45-64,BLACK,M,41,-74,40.87-73.92,UNKNOWN,UNKNOWN,E,40.87-73.92,MN01,Marble Hill-Inwood,36061029900
1043972,743292142,12/31/2018,19:41:00,12/31/2018,341,PETIT LARCENY,338,"LARCENY,PETIT FROM BUILDING,UN",MISDEMEANOR,QUEENS,INSIDE,LIQUOR STORE,1015653,194856,UNKNOWN,WHITE,F,41,-74,40.70-73.89,UNKNOWN,UNKNOWN,D,40.70-73.89,QN99,park-cemetery-etc-Queens,36081056100
1043973,439692376,12/31/2018,06:20:00,12/31/2018,578,HARRASSMENT 2,638,"HARASSMENT,SUBD 3,4,5",VIOLATION,BROOKLYN,INSIDE,OTHER,1000401,192939,25-44,BLACK,M,41,-74,40.70-73.94,18-24,BLACK,M,40.70-73.94,BK75,Bedford,36047025700
1043974,637872275,12/31/2018,20:10:00,12/31/2018,105,ROBBERY,379,"ROBBERY,GAS STATION",FELONY,QUEENS,INSIDE,GAS STATION,1050202,193549,UNKNOWN,BLACK,M,41,-74,40.70-73.76,UNKNOWN,UNKNOWN,D,40.70-73.76,QN08,St. Albans,36081052600


In [239]:
del df_s['latlong']

In [234]:
df_s.dropna(subset = ['latlong'], inplace = True)

In [237]:
df_s = df_s.reset_index(drop = True)

## Merge Census Data with Crime Data

In [242]:
census = pd.read_csv('Data/new-york-city-census-data/nyc_census_tracts.csv')
census_s = census.loc[:,['CensusTract', 'TotalPop', 'Men', 'Women','Hispanic', 'White', 'Black', 'Native', 
 'Asian', 'Citizen', 'Income','IncomePerCap', 'Poverty','MeanCommute', 'Unemployment']]
census_s

,CensusTract,TotalPop,Men,Women,Hispanic,White,Black,Native,Asian,Citizen,Income,IncomePerCap,Poverty,MeanCommute,Unemployment
0,36005000100,7703,7133,570,30,6,61,0,2,6476,nan,2440,nan,nan,nan
1,36005000200,5403,2659,2744,76,2,16,0,4,3639,72034,22180,20,43,8
2,36005000400,5915,2896,3019,63,4,31,0,0,4100,74836,27700,13,45,10
3,36005001600,5879,2558,3321,65,2,32,0,0,3536,32312,17526,26,39,9
4,36005001900,2591,1206,1385,55,9,29,0,2,1557,37936,17986,37,45,19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2162,36085030302,6279,3093,3186,36,29,18,0,14,3842,76542,25189,19,47,6
2163,36085031901,2550,953,1597,27,6,60,0,6,1558,25064,12753,39,38,14
2164,36085031902,4611,2043,2568,21,15,62,0,1,2649,22656,16303,41,41,9
2165,36085032300,1131,597,534,46,24,30,0,0,768,56406,34261,20,52,11


In [244]:
census_s.columns

Index(['CensusTract', 'TotalPop', 'Men', 'Women', 'Hispanic', 'White', 'Black',
       'Native', 'Asian', 'Citizen', 'Income', 'IncomePerCap', 'Poverty',
       'MeanCommute', 'Unemployment'],
      dtype='object')

In [243]:
census_s['CensusTract'] = census_s['CensusTract'].apply(lambda x : str(x))

In [247]:
crime_df = pd.merge(df_s, census_s[['CensusTract','TotalPop', 'Men', 'Women','Income','Poverty','Unemployment']], how = 'left', left_on = 'BlockCode', right_on = 'CensusTract' )

In [249]:
crime_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1043938 entries, 0 to 1043937
Data columns (total 33 columns):
CMPLNT_NUM           1043938 non-null int64
CMPLNT_FR_DT         1043938 non-null object
CMPLNT_FR_TM         1043938 non-null object
RPT_DT               1043938 non-null object
KY_CD                1043938 non-null int64
OFNS_DESC            1043882 non-null object
PD_CD                1043938 non-null float64
PD_DESC              1043938 non-null object
LAW_CAT_CD           1043938 non-null object
BORO_NM              1043938 non-null object
LOC_OF_OCCUR_DESC    841119 non-null object
PREM_TYP_DESC        1040053 non-null object
X_COORD_CD           1043938 non-null float64
Y_COORD_CD           1043938 non-null float64
SUSP_AGE_GROUP       758673 non-null object
SUSP_RACE            758673 non-null object
SUSP_SEX             758673 non-null object
Latitude             1043938 non-null float64
Longitude            1043938 non-null float64
Lat_Lon              1043938 non-

In [250]:
with open('crime_df.pkl', 'wb') as picklefile:
    pickle.dump(crime_df, picklefile)